In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

from tests.sama4d.video_to_tracks.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [3]:
scene_spec = get_cheezitbox_scene_loader()
task = KeypointTrackingTask(scene_spec["feature_track_data_loader"], scene_name=scene_spec["scene_name"], n_frames=10)

In [4]:
rr.init(f'patch_tracker_port-2')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [5]:
import b3d.chisight.particle_system as ps
import b3d.chisight.particle_system_patch_tracking as pt
import b3d.chisight.dense.likelihoods as likelihoods
from b3d.chisight.dense.differentiable_renderer import DifferentiableRendererHyperparams
from b3d import Pose
import b3d
import genjax
from genjax import ChoiceMapBuilder as C
import jax
import jax.numpy as jnp
genjax.pretty()

In [6]:
task_spec = task.get_task_specification()
video = task_spec["video"]
assert video.shape[-1] == 4, "Expected video to be RGBD"
poses_WC = task_spec["poses_WC"]
initial_patch_centers_2D = task_spec["initial_keypoint_positions_2D"]
r = task_spec["renderer"]
fx, fy, cx, cy = r.fx, r.fy, r.cx, r.cy
renderer : b3d.Renderer = r

In [54]:
model = pt.get_default_multiobject_model_for_patchtracking(renderer)

In [55]:
(patches, poses_CP_at_time0, patch_points_C) = pt.get_patches(
    initial_patch_centers_2D, video, Pose.identity(), fx, fy, cx, cy
)

In [56]:
(get_initial_tracker_state, update_tracker_state, get_trace) = pt.get_adam_optimization_patch_tracker(model, patches, pose_WC=Pose.identity())

In [57]:
st = get_initial_tracker_state(poses_CP_at_time0)

In [58]:
(pos, quat), st2 = update_tracker_state(st, video[1])

In [59]:
trace = get_trace(pos, quat, video[1])
trace.get_score()

<jax.Array(-24900.598, dtype=float32)>

In [60]:
trace2 = get_trace(pos, quat, video[10])
trace2.get_score()

<jax.Array(-43576.2, dtype=float32)>

In [29]:
jnp.all(trace.get_choices()["obs", "image"] == trace2.get_choices()["obs", "image"])

<jax.Array(False, dtype=bool)>

In [33]:
trace.subtraces[1].score

<jax.Array(-44266.56, dtype=float32)>

In [34]:
trace2.subtraces[1].score

<jax.Array(-44266.56, dtype=float32)>

In [38]:
jnp.all(trace2.subtraces[1].subtraces[0].value == trace.subtraces[1].subtraces[0].value)

<jax.Array(False, dtype=bool)>

In [39]:
trace2.subtraces[1].subtraces[0].score

<jax.Array(-44266.56, dtype=float32)>

In [40]:
trace.subtraces[1].subtraces[0].score

<jax.Array(-44266.56, dtype=float32)>

In [ ]:
trace.get_args

In [48]:
particle_dynamics_summary, final_state = trace.subtraces[0].get_retval()
absolute_particle_poses_last_frame = particle_dynamics_summary["absolute_particle_poses"][-1]
camera_pose_last_frame = particle_dynamics_summary["camera_pose"][-1]
absolute_particle_poses_in_camera_frame = camera_pose_last_frame.inv() @ absolute_particle_poses_last_frame
absolute_particle_poses_in_camera_frame

Pose(position=Array([[ 4.3720007e-01,  8.0211794e-01, -8.5780412e-01],
       [ 2.0888785e-01,  5.0581485e-01, -8.3555418e-01],
       [-8.6002991e-02, -7.0305049e-02,  1.6450262e+00],
       [ 1.9970134e+00, -2.6645586e-01, -3.7854290e-01],
       [-4.1008365e-01,  9.2307913e-01,  5.8423245e-01],
       [ 4.7509372e-01,  1.2065130e+00,  1.2806077e+00],
       [ 1.7191978e+00, -2.9092574e-01, -9.3487933e-02],
       [-1.9009495e-01, -6.8191260e-02, -1.0819732e+00],
       [-4.2093849e-01, -6.9401354e-01,  1.2477970e+00],
       [-4.3289030e-01, -3.0505949e-01, -1.0283262e+00],
       [-5.8384925e-01,  1.5033858e-01, -1.4844725e+00],
       [ 1.4291054e-01, -4.3197721e-01,  1.4950497e+00],
       [ 5.3572321e-01, -1.9140496e+00,  8.4339726e-01],
       [-3.7687075e-01, -5.1340389e-01, -6.3229001e-01],
       [ 6.7030311e-01, -2.7323678e-01,  1.0744183e+00],
       [ 6.7458469e-01,  5.7633483e-01, -9.7053871e-04],
       [-1.8737086e+00, -1.4755149e+00,  1.2432760e-01],
       [-2.7000985e-01, -1.2513275e+00, -1.0954665e+00],
       [ 5.0870996e-02, -1.8179338e+00,  5.7380813e-01],
       [ 2.7167615e-01, -5.4833478e-01, -7.0832068e-01],
       [-9.4845563e-01, -7.0319623e-02, -1.3413935e+00],
       [-5.9046316e-01, -7.5977731e-01,  1.5933509e+00],
       [-1.3145757e-01,  1.0122905e+00,  1.1007499e+00],
       [ 8.3642161e-01, -6.9723815e-01,  1.0305436e+00],
       [-1.9640998e+00, -1.2670376e+00,  3.1705588e-01],
       [ 1.0717390e+00,  6.8036747e-01, -7.0959193e-01],
       [-8.6124283e-01,  4.2587894e-01, -7.9391086e-01],
       [-6.6353840e-01,  1.1505849e+00,  6.4018142e-01],
       [-1.4677417e-01, -4.6814787e-01, -8.1121826e-01],
       [-3.1878477e-01, -4.4097996e-01, -4.4316107e-01],
       [ 1.1028324e+00, -7.9922181e-01,  7.2457157e-03],
       [ 1.4379355e+00,  9.4439727e-01, -6.4321935e-01],
       [-1.3249424e+00,  5.8260447e-01, -8.0199116e-01],
       [-9.3055534e-01, -8.1348985e-01,  1.0473322e+00],
       [-5.8843249e-01,  3.7301433e-01, -7.5018011e-02],
       [ 2.0679955e+00, -2.9724874e-02, -4.9640584e-01],
       [-1.1051048e-02,  1.3392069e+00, -5.5422181e-01],
       [ 2.1405581e-01, -1.4488869e+00,  2.7260643e-01],
       [-1.7481713e-01,  2.1384180e-01, -1.0102755e+00],
       [ 1.2082536e+00, -7.4631470e-01, -9.2654413e-01],
       [ 2.8111902e-01,  1.3365961e+00, -1.4432968e+00],
       [ 2.6064509e-01, -6.8983084e-01,  1.8607497e+00],
       [-3.2630962e-01, -1.5376788e+00, -8.8850248e-01],
       [ 6.4072829e-01, -2.1656065e-01,  1.2937351e+00],
       [ 1.0217810e+00,  9.9908400e-01, -4.8633617e-01],
       [ 2.1138134e+00, -2.7474672e-01, -6.5263337e-01],
       [ 5.9225696e-01, -2.2849999e-01,  1.2008866e+00],
       [ 3.4968708e-02, -1.6592637e+00, -4.3137449e-01]], dtype=float32), quaternion=Array([[-0.08408704,  0.6015122 , -0.3847217 , -0.6950551 ],
       [-0.33257684,  0.3776463 , -0.5238242 , -0.6873022 ],
       [ 0.5696879 ,  0.3260746 ,  0.5169495 , -0.5494492 ],
       [ 0.08277787,  0.00615881, -0.34064108, -0.9365221 ],
       [ 0.34015143, -0.11222337,  0.50259644, -0.786829  ],
       [ 0.6184576 , -0.25312638, -0.5082411 , -0.543257  ],
       [-0.06354008,  0.03217747, -0.50723493, -0.8588597 ],
       [ 0.60902476,  0.2394621 , -0.45399633, -0.6046768 ],
       [ 0.17309356,  0.80335   , -0.04573532, -0.5679576 ],
       [ 0.46749976,  0.48622507, -0.32414082, -0.6632963 ],
       [ 0.3554105 ,  0.34943464, -0.71705985, -0.4872413 ],
       [ 0.49656618,  0.09970935, -0.00119225, -0.8622522 ],
       [-0.21102627, -0.5578959 , -0.5226564 , -0.60913914],
       [ 0.6451167 , -0.01995095,  0.22538312, -0.72981435],
       [-0.5515673 , -0.08369855,  0.20456482, -0.80431426],
       [-0.26964563,  0.26520127, -0.5104361 , -0.7722788 ],
       [-0.14527649,  0.38875252, -0.30544558, -0.8570118 ],
       [-0.5143998 ,  0.522504  , -0.01698808, -0.6797749 ],
       [-0.5940443 ,  0.01886644, -0.67264336, -0.440802  ],
       [ 0.31963456, -0.09414306, -0.39808103, -0.8546943 ],
  

In [51]:
camera_pose_last_frame

Pose(position=Array([0.05339679, 0.0229879 , 0.04690184], dtype=float32), quaternion=Array([-0.02302795,  0.03529329, -0.1070694 ,  0.9933581 ], dtype=float32))

In [53]:
particle_dynamics_summary

{'absolute_particle_poses': Pose(position=Array([[[ 0.58442634,  0.6793925 , -0.8791141 ],
         [ 0.30104047,  0.4399839 , -0.8262651 ],
         [ 0.07819659,  0.0356308 ,  1.6954392 ],
         [ 1.9149926 , -0.67957735, -0.44627085],
         [-0.10756323,  1.0341352 ,  0.60658395],
         [ 0.86720645,  1.1476356 ,  1.227675  ],
         [ 1.6604663 , -0.63302577, -0.14281407],
         [-0.2274616 , -0.04414534, -1.0152016 ],
         [-0.40969607, -0.51649636,  1.3547027 ],
         [-0.51005995, -0.22124135, -0.9332912 ],
         [-0.5952845 ,  0.238168  , -1.4022532 ],
         [ 0.21369006, -0.37215918,  1.5503533 ],
         [ 0.23475987, -1.9277635 ,  0.9544151 ],
         [-0.4697219 , -0.42146975, -0.5312071 ],
         [ 0.7296176 , -0.34633392,  1.0883749 ],
         [ 0.83242047,  0.4408685 , -0.02876218],
         [-2.0748215 , -1.0107691 ,  0.37158474],
         [-0.5561073 , -1.1822864 , -0.9603673 ],
         [-0.23771113, -1.7403255 ,  0.7122656 ],
         [ 0.14925997, -0.5974773 , -0.6473822 ],
         [-0.986464  ,  0.10641799, -1.2241521 ],
         [-0.5628603 , -0.5311488 ,  1.7135856 ],
         [ 0.22157627,  1.0812123 ,  1.0983485 ],
         [ 0.79871786, -0.7974461 ,  1.0564297 ],
         [-2.1044433 , -0.7805576 ,  0.55840737],
         [ 1.1882588 ,  0.43021485, -0.7663015 ],
         [-0.75563705,  0.5929299 , -0.71075034],
         [-0.302352  ,  1.3126464 ,  0.66672826],
         [-0.2493494 , -0.43345237, -0.72691154],
         [-0.38363045, -0.35601008, -0.3503971 ],
         [ 0.9600316 , -0.9931685 ,  0.0248366 ],
         [ 1.6058623 ,  0.6119547 , -0.7381106 ],
         [-1.1750729 ,  0.84497744, -0.69692963],
         [-0.94662404, -0.5315306 ,  1.1945391 ],
         [-0.44696838,  0.5103149 , -0.00937635],
         [ 2.0252967 , -0.46824104, -0.5809618 ],
         [ 0.28372747,  1.3112712 , -0.5760211 ],
         [-0.02337669, -1.4266123 ,  0.38182324],
         [-0.14765668,  0.23058584, -0.95978904],
         [ 1.0038614 , -0.9997903 , -0.915329  ],
         [ 0.50119567,  1.2121428 , -1.4808446 ],
         [ 0.3014467 , -0.6350959 ,  1.9208252 ],
         [-0.65588975, -1.4417968 , -0.73520374],
         [ 0.7292179 , -0.276302  ,  1.3058194 ],
         [ 1.2236037 ,  0.76052064, -0.5575722 ],
         [ 2.0065033 , -0.7231726 , -0.72655964],
         [ 0.6724901 , -0.2811116 ,  1.2170967 ],
         [-0.29515326, -1.6204433 , -0.2967685 ]]], dtype=float32), quaternion=Array([[[ 0.01669738, -0.5731301 ,  0.3186312 ,  0.7547963 ],
         [ 0.29259384, -0.37442702,  0.44371462,  0.7598096 ],
         [-0.6317142 , -0.2554251 , -0.5447301 ,  0.4888398 ],
         [-0.09243133,  0.04364228,  0.24116905,  0.96508515],
         [-0.3617338 ,  0.1640938 , -0.57408273,  0.7159965 ],
         [-0.58182037,  0.34854013,  0.46269768,  0.5708903 ],
         [ 0.05779704,  0.00322562,  0.41040677,  0.91006345],
         [-0.62852025, -0.14086808,  0.4132474 ,  0.6436965 ],
         [-0.26942283, -0.75838286,  0.00922922,  0.593449  ],
         [-0.5202888 , -0.40206647,  0.27866554,  0.69999117],
         [-0.37637636, -0.27535135,  0.6807189 ,  0.56492853],
         [-0.52375764, -0.01542087, -0.07131494,  0.84873694],
         [ 0.27377728,  0.5651302 ,  0.43366975,  0.64622325],
         [-0.6634564 ,  0.10945813, -0.27971807,  0.68527544],
         [ 0.5311239 ,  0.04776282, -0.31071755,  0.7868168 ],
         [ 0.2396907 , -0.25330007,  0.4209488 ,  0.83737063],
         [ 0.09373304, -0.3644446 ,  0.21548203,  0.9010893 ],
         [ 0.43998477, -0.5497274 , -0.0620305 ,  0.7073652 ],
         [ 0.60166776, -0.05129816,  0.60044795,  0.52423924],
         [-0.31306398,  0.16707282,  0.31303844,  0.8809566 ],
         [-0.23739186, -0.37821287,  0.6758834 ,  0.58632904],
         [-0.18456535, -0.6898721 ,  0.01417511,  0.69986516],
         [ 0.48831135,  0.25601113, -0.10662142,  0.82743114],
         [ 0.62025565, -0.1413602 , -0.5113157 ,  0.57780313],
         [ 0.176374

In [30]:
trace

<unknown>:24: SyntaxWarning: invalid escape sequence '\p'
<unknown>:172: SyntaxWarning: invalid escape sequence '\p'


/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:398: RuntimeWarning: divide by zero encountered in log2
  + doubling_bonus * np.log2(sorted_shape / cutoff_size_per_axis),
/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:410: RuntimeWarning: divide by zero encountered in log
  log_axis_proportions = np.log(axis_proportions)
/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:420: RuntimeWarning: invalid value encountered in scalar add
  soft_limit_for_this_axis = np.exp(log_c + log_axis_proportions[i])


ValueError: cannot convert float NaN to integer

/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/renderer.py:256: UserWarning: Ignoring error inside wrapper hook <function use_autovisualizer_if_present at 0x7fd1e1ed9440>:
Traceback (most recent call last):
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/renderer.py", line 231, in _render_subtree
    postprocessed_result = hook(
                           ^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/autovisualize.py", line 154, in use_autovisualizer_if_present
    result = autoviz(node, path)
             ^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/arrayviz/array_autovisualizer.py", line 358, in __call__
    renderable=foldable_impl.maybe_defer_rendering(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/foldable_representation/foldable_impl.py", line 653, in maybe_defer_re

StaticTrace(...)

In [31]:
trace2

/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:398: RuntimeWarning: divide by zero encountered in log2
  + doubling_bonus * np.log2(sorted_shape / cutoff_size_per_axis),
/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:410: RuntimeWarning: divide by zero encountered in log
  log_axis_proportions = np.log(axis_proportions)
/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/ndarray_summarization.py:420: RuntimeWarning: invalid value encountered in scalar add
  soft_limit_for_this_axis = np.exp(log_c + log_axis_proportions[i])


ValueError: cannot convert float NaN to integer

/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/renderer.py:256: UserWarning: Ignoring error inside wrapper hook <function use_autovisualizer_if_present at 0x7fd1e1ed9440>:
Traceback (most recent call last):
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/renderer.py", line 231, in _render_subtree
    postprocessed_result = hook(
                           ^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/autovisualize.py", line 154, in use_autovisualizer_if_present
    result = autoviz(node, path)
             ^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/arrayviz/array_autovisualizer.py", line 358, in __call__
    renderable=foldable_impl.maybe_defer_rendering(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/b3d-2/lib/python3.12/site-packages/penzai/treescope/foldable_representation/foldable_impl.py", line 653, in maybe_defer_re

StaticTrace(...)

In [8]:
depth_scale, color_scale, mindepth, maxdepth = 0.0001, 0.002, -20.0, 20.0
likelihood = likelihoods.get_uniform_multilaplace_image_dist_with_fixed_params(
        renderer.height, renderer.width, depth_scale, color_scale, mindepth, maxdepth
)
@genjax.gen
def wrapped_likelihood(mesh : b3d.Mesh, args):
    weights, attributes = b3d.chisight.dense.differentiable_renderer.render_to_rgbd_dist_params(
        renderer, mesh.vertices, mesh.faces, mesh.vertex_attributes,
        DifferentiableRendererHyperparams(3, 1e-5, 1e-2, -1)
    )
    obs = likelihood(weights, attributes) @ "image"
    return obs, {"diffrend_output": (weights, attributes)}

model = ps.make_dense_gps_model(wrapped_likelihood)

In [29]:
max_num_timesteps = genjax.Pytree.const(1)
num_particles = genjax.Pytree.const(patches.vertices.shape[0])
num_clusters = num_particles
relative_particle_poses_prior_params = (Pose.identity(), .5, 0.25)
initial_object_poses_prior_params = (Pose.identity(), 2., 0.5)
camera_pose_prior_params = (Pose.identity(), 0.1, 0.1)
instrinsics = genjax.Pytree.const(renderer.get_intrinsics_object())
sigma_obs = 0.2

model_args = (
    (
        max_num_timesteps, # const object
        num_particles, # const object
        num_clusters, # const object
        relative_particle_poses_prior_params,
        initial_object_poses_prior_params,
        camera_pose_prior_params
    ),
    (patches, ())
)

In [30]:
def allidx_chm(x):
    return genjax.ChoiceMap.idx(jnp.arange(x.shape[0], dtype=int), x)

In [31]:
b3d.modeling_utils.bernoulli.sample(jax.random.PRNGKey(10), (-0.1,))

<jax.Array([1], dtype=int32)>

In [32]:
image = video[0]
particle_poses = jax.tree.map(
    lambda arr: jnp.tile(arr, (num_particles.const, 1)),
    Pose.identity()
)
pose_WC = poses_WC[0]
object_assignments = jnp.arange(num_particles.const, dtype=int)
object_poses = poses_CP_at_time0
vis_mask = jnp.ones((num_particles.const,), dtype=int)

dc = C["state0"].set(C.d({
    "particle_poses": allidx_chm(particle_poses),
    "object_assignments": allidx_chm(object_assignments),
    "object_poses": allidx_chm(object_poses),
    "initial_camera_pose": pose_WC,
    "initial_visibility": allidx_chm(vis_mask),
}))
dc

StaticChm(...)

In [33]:
constraints = C.d({
    "particle_dynamics": dc,
    "obs": C["image"].set(image),
})

constraints

XorChm(...)

In [34]:
trace, weight = model.importance(
    jax.random.PRNGKey(200),
    constraints,
    model_args
)

In [27]:
trace.get_choices()("obs")("image")

ValueChm(...)

In [21]:
weight

<jax.Array(-44255.61, dtype=float32)>

In [42]:
ps.visualize_particle_system(
    trace.get_args()[0], trace.get_retval()[0], trace.get_retval()[1],
    transform_Viz_World=pose_WC
)